## Dependencies

In [1]:
import re
import warnings
import pandas as pd
from joblib import load
import tensorflow_hub as hub
from tensorflow.keras import Model
from tensorflow.keras.layers import Lambda, Input, Dense, Dropout, Concatenate, Average
from googleqa_utilityscript import *

SEED = 0
seed_everything(SEED)
warnings.filterwarnings("ignore")

## Load data

In [2]:
module_url = '/kaggle/input/universalsentenceencodermodels/universal-sentence-encoder-models/use'
model_path = '/kaggle/input/41-googleq-a-train-use-lecun-normal2/use.h5'
# category_encoder_path = '/kaggle/input/39-googleq-a-train-use-glorot-uniform/category_encoder.joblib'

test = pd.read_csv('/kaggle/input/google-quest-challenge/test.csv')

print('Test samples: %s' % len(test))
display(test.head())

Test samples: 476


,qa_id,question_title,question_body,question_user_name,question_user_page,answer,answer_user_name,answer_user_page,url,category,host
0,39,Will leaving corpses lying around upset my pri...,I see questions/information online about how t...,Dylan,https://gaming.stackexchange.com/users/64471,There is no consequence for leaving corpses an...,Nelson868,https://gaming.stackexchange.com/users/97324,http://gaming.stackexchange.com/questions/1979...,CULTURE,gaming.stackexchange.com
1,46,Url link to feature image in the portfolio,I am new to Wordpress. i have issue with Featu...,Anu,https://wordpress.stackexchange.com/users/72927,I think it is possible with custom fields.\n\n...,Irina,https://wordpress.stackexchange.com/users/27233,http://wordpress.stackexchange.com/questions/1...,TECHNOLOGY,wordpress.stackexchange.com
2,70,"Is accuracy, recoil or bullet spread affected ...","To experiment I started a bot game, toggled in...",Konsta,https://gaming.stackexchange.com/users/37545,You do not have armour in the screenshots. Thi...,Damon Smithies,https://gaming.stackexchange.com/users/70641,http://gaming.stackexchange.com/questions/2154...,CULTURE,gaming.stackexchange.com
3,132,Suddenly got an I/O error from my external HDD,I have used my Raspberry Pi as a torrent-serve...,robbannn,https://raspberrypi.stackexchange.com/users/17341,Your Western Digital hard drive is disappearin...,HeatfanJohn,https://raspberrypi.stackexchange.com/users/1311,http://raspberrypi.stackexchange.com/questions...,TECHNOLOGY,raspberrypi.stackexchange.com
4,200,Passenger Name - Flight Booking Passenger only...,I have bought Delhi-London return flights for ...,Amit,https://travel.stackexchange.com/users/29089,I called two persons who work for Saudia (tick...,Nean Der Thal,https://travel.stackexchange.com/users/10051,http://travel.stackexchange.com/questions/4704...,CULTURE,travel.stackexchange.com


In [3]:
question_target_cols = ['question_asker_intent_understanding','question_body_critical', 'question_conversational', 
                        'question_expect_short_answer', 'question_fact_seeking', 'question_has_commonly_accepted_answer',
                        'question_interestingness_others', 'question_interestingness_self', 'question_multi_intent', 
                        'question_not_really_a_question', 'question_opinion_seeking', 'question_type_choice',
                        'question_type_compare', 'question_type_consequence', 'question_type_definition', 
                        'question_type_entity', 'question_type_instructions', 'question_type_procedure',
                        'question_type_reason_explanation', 'question_type_spelling', 'question_well_written']
answer_target_cols = ['answer_helpful', 'answer_level_of_information', 'answer_plausible', 'answer_relevance',
                      'answer_satisfaction', 'answer_type_instructions', 'answer_type_procedure', 
                      'answer_type_reason_explanation', 'answer_well_written']
target_cols = question_target_cols + answer_target_cols

## Pre-process data

In [4]:
text_features = ['question_title', 'question_body', 'answer']

for feature in text_features:
    # Lower
    test[feature] = test[feature].apply(lambda x: x.lower())
    # Map misspellings
#     test[feature] = test[feature].apply(lambda x: map_misspellings(x))
    # Map contractions
    test[feature] = test[feature].apply(lambda x: map_contraction(x))
    # Trim text
    test[feature] = test[feature].apply(lambda x: x.strip())

## Test set

In [5]:
# ohe_category = load(category_encoder_path)

# Test features
X_test_title = test['question_title']
X_test_body = test['question_body']
X_test_answer = test['answer']
# X_test_category = ohe_category.transform(test[['category']]).toarray()

X_test = [X_test_title, X_test_body, X_test_answer]

# Model parameters

In [6]:
EMBEDDDING_SIZE = 512
N_CLASS = len(target_cols)

# Model

In [7]:
use_embed = hub.load(module_url)

def USEEmbedding(x):
    return use_embed(tf.squeeze(tf.cast(x, tf.string)))

def encoder_block(input_layer):
    encoder = Lambda(USEEmbedding, output_shape=(EMBEDDDING_SIZE,))(input_layer)
    return encoder

In [8]:
# Title
input_title = Input(shape=(1,), dtype=tf.string, name='input_title')
encoded_title = encoder_block(input_title)

# Body
input_body = Input(shape=(1,), dtype=tf.string, name='input_body')
encoded_body = encoder_block(input_body)

# Answer
input_answer = Input(shape=(1,), dtype=tf.string, name='input_answer')
encoded_answer = encoder_block(input_answer)

# Output
x = Concatenate()([encoded_title, encoded_body, encoded_answer])
x = Dropout(0.5)(x)
x = Dense(512, activation='relu', kernel_initializer='lecun_normal')(x)
x = Dropout(0.5)(x)
output = Dense(N_CLASS, activation='sigmoid', kernel_initializer='lecun_normal', name='output')(x)
model = Model(inputs=[input_title, input_body, input_answer], outputs=[output])

model.summary()

model.load_weights(model_path)

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_title (InputLayer)        [(None, 1)]          0                                            
__________________________________________________________________________________________________
input_body (InputLayer)         [(None, 1)]          0                                            
__________________________________________________________________________________________________
input_answer (InputLayer)       [(None, 1)]          0                                            
__________________________________________________________________________________________________
lambda (Lambda)                 (None, 512)          0           input_title[0][0]                
______________________________________________________________________________________________

# Make predictions

In [9]:
Y_test = model.predict(X_test)

In [10]:
submission = pd.read_csv('/kaggle/input/google-quest-challenge/sample_submission.csv')
submission[target_cols] = Y_test
submission.to_csv("submission.csv", index=False)
display(submission.head())

,qa_id,question_asker_intent_understanding,question_body_critical,question_conversational,question_expect_short_answer,question_fact_seeking,question_has_commonly_accepted_answer,question_interestingness_others,question_interestingness_self,question_multi_intent,...,question_well_written,answer_helpful,answer_level_of_information,answer_plausible,answer_relevance,answer_satisfaction,answer_type_instructions,answer_type_procedure,answer_type_reason_explanation,answer_well_written
0,39,0.887256,0.612371,0.251142,0.653585,0.564595,0.687449,0.619348,0.573867,0.322523,...,0.876380,0.904787,0.617650,0.945138,0.948564,0.828976,0.116400,0.066221,0.814911,0.914148
1,46,0.857713,0.499630,0.005242,0.661789,0.845037,0.878936,0.564374,0.462629,0.204160,...,0.717484,0.910560,0.598375,0.951030,0.953624,0.807814,0.883672,0.121358,0.098369,0.837901
2,70,0.877696,0.625087,0.049555,0.759004,0.893779,0.889658,0.589668,0.491397,0.322430,...,0.839409,0.889954,0.592976,0.946650,0.942050,0.805715,0.101544,0.114262,0.758363,0.879427
3,132,0.862887,0.479750,0.009195,0.687119,0.749220,0.869276,0.530036,0.443693,0.260527,...,0.714774,0.931185,0.656648,0.963516,0.972080,0.855293,0.807323,0.131764,0.685778,0.884896
4,200,0.965752,0.587142,0.020071,0.884634,0.767396,0.906777,0.676580,0.659686,0.368811,...,0.827086,0.967729,0.710438,0.988260,0.986197,0.912241,0.122494,0.130364,0.866232,0.940262
